# Gateway Trading Router Example

This notebook demonstrates how to use the Gateway Trading router for DEX swap operations.

The Gateway Trading router provides functionality for:
- Getting swap quotes (price discovery)
- Executing swaps on DEX routers (Jupiter, 0x)

## Setup

Initialize the Hummingbot API client.

In [1]:
from hummingbot_api_client import HummingbotAPIClient
from decimal import Decimal

client = HummingbotAPIClient(base_url='http://localhost:8000')
await client.init()

## Get Swap Quote

Get a price quote for a swap without executing the transaction.

In [2]:
# Get quote for buying 1 SOL with USDC on Jupiter
quote = await client.gateway_trading.get_swap_quote(
    connector='jupiter',
    network='solana-mainnet-beta',
    trading_pair='SOL-USDC',
    side='BUY',
    amount=Decimal('1'),
    slippage_pct=Decimal('1.0')  # 1% slippage
)
quote

{'base': 'SOL',
 'quote': 'USDC',
 'price': '179.644109',
 'amount': '1',
 'amount_in': '179.644109',
 'amount_out': '1',
 'expected_amount': '1',
 'slippage_pct': '1.0',
 'gas_estimate': None}

In [3]:
# Display quote information
print(f"Trading Pair: {quote['base']}/{quote['quote']}")
print(f"Price: ${quote['price']}")
print(f"Amount In: {quote['amount_in']} {quote['quote']}")
print(f"Amount Out: {quote['amount_out']} {quote['base']}")
print(f"Slippage: {quote['slippage_pct']}%")
if quote.get('gas_estimate'):
    print(f"Gas Estimate: {quote['gas_estimate']}")

Trading Pair: SOL/USDC
Price: $179.644109
Amount In: 179.644109 USDC
Amount Out: 1 SOL
Slippage: 1.0%


In [4]:
# Get quote for selling 1 SOL for USDC
quote_sell = await client.gateway_trading.get_swap_quote(
    connector='jupiter',
    network='solana-mainnet-beta',
    trading_pair='SOL-USDC',
    side='SELL',
    amount=Decimal('1'),
    slippage_pct=Decimal('1.0')
)

print(f"Selling 1 SOL:")
print(f"You will receive: ~{quote_sell['amount_out']} USDC")
print(f"Price: ${quote_sell['price']} per SOL")

Selling 1 SOL:
You will receive: ~179.666619 USDC
Price: $179.666619 per SOL


## Swap with Custom Tokens (LIGHT Example)

Get quotes for swaps involving the custom token we added earlier.

In [5]:
# Get quote for buying GOLD with USDC
# Note: Make sure you added GOLD token first using the gateway router
quote_gold = await client.gateway_trading.get_swap_quote(
    connector='jupiter',
    network='solana-mainnet-beta',
    trading_pair='9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump-USDC',
    side='BUY',
    amount=Decimal('100'),  # Buy 100 GOLD
    slippage_pct=Decimal('5.0')  # 2% slippage for smaller tokens
)

print(f"Buying 100 FARTCOIN:")
print(f"You will pay: ~{quote_gold['amount_in']} USDC")
print(f"Price: ${quote_gold['price']} per GOLD")

Buying 100 FARTCOIN:
You will pay: ~35.890788 USDC
Price: $0.35890788 per GOLD


## Example Workflow: Quote and Execute

Complete workflow showing how to get a quote and then execute if the price is acceptable.

In [7]:
# 1. Get quote
print("Step 1: Getting quote...")
quote = await client.gateway_trading.get_swap_quote(
    connector='jupiter',
    network='solana-mainnet-beta',
    trading_pair='FARTCOIN-USDT',
    side='BUY',
    amount=Decimal('1'),
    slippage_pct=Decimal('1.0')
)

print(f"Quote received:")
print(f"  Price: ${quote['price']}")
print(f"  You will pay: {quote['amount_in']} USDC")
print(f"  You will receive: {quote['amount_out']} SOL")

# 2. Check if price is acceptable
max_price = Decimal('200')  # Maximum price you're willing to pay
quote_price = Decimal(str(quote['price']))

print(f"\nStep 2: Checking price...")
if quote_price <= max_price:
    print(f"✅ Price ${quote_price} is acceptable (max: ${max_price})")
    print("\nStep 3: Ready to execute swap")

    # 3. Execute swap (uncomment to run)
    result = await client.gateway_trading.execute_swap(
        connector='jupiter',
        network='solana-mainnet-beta',
        trading_pair='FARTCOIN-USDT',
        side='BUY',
        amount=Decimal('1'),
        slippage_pct=Decimal('1.0')
    )
    print(f"\n✅ Swap executed!")
    print(f"Transaction: {result['transaction_hash']}")
else:
    print(f"❌ Price ${quote_price} is too high (max: ${max_price})")
    print("Swap cancelled")

Step 1: Getting quote...
Quote received:
  Price: $0.358422
  You will pay: 0.358422 USDC
  You will receive: 1 SOL

Step 2: Checking price...
✅ Price $0.358422 is acceptable (max: $200)

Step 3: Ready to execute swap

✅ Swap executed!
Transaction: mRVrqADMvxT4Yv2GmRnCpouTDeBi7FqkzLVrgRwbwEE5kcxH47TNGwHKeEuAPqXDaTtRot7J6WCsL2of2VyMLzQ


## Using Other DEX Connectors

The same methods work with other DEX routers like 0x on Ethereum networks.

In [ ]:
# Example: Get quote on Ethereum using 0x
# quote_eth = await client.gateway_trading.get_swap_quote(
#     connector='0x',
#     network='ethereum-mainnet',
#     trading_pair='ETH-USDC',
#     side='BUY',
#     amount=Decimal('1'),
#     slippage_pct=Decimal('1.0')
# )
# quote_eth

## Tips and Best Practices

1. **Always get a quote first** - Check the price and expected amounts before executing
2. **Set appropriate slippage** - Higher slippage for volatile/illiquid tokens (2-5%), lower for stable pairs (0.5-1%)
3. **Check gas estimates** - Make sure you have enough native tokens for gas fees
4. **Use wallet_address parameter** - Specify which wallet to use if you have multiple configured
5. **Monitor transaction status** - Save the transaction hash to verify on blockchain explorers
6. **Start with small amounts** - Test with small trades before executing large swaps